In [1]:
from kdp import *

Let's build a graph from a region that's super simple (hom, both matching) first.

We'll have the H (haplotypes (or reads) graph) and G (graph variants to be phased/genotyped).

Starting at the snk of H - we'll have two options, use it, or don't use it.

For the use it, we check every neighboring node for G from snk and record the sizediff and cossim.
    if the sizediff and cossim is above thresholds,
        We apply Hn to Gn, update our running Hsk and Gsk (size and kmer features of the paths being joined).

if we can't use it, well.. because we have in the graph snk->everyNode, we know we've checked Use it and don't use it.
So, not using it is just saying, 'well go to the next node'.

Okay, we we have two things.
    Hi and Gi
    And we try to apply each Hi+1 to Gi+1,
    If none of them are good enough... we need to try and both Hi+1 and Hi+2 as well as just moving to Hi+2.
    That's the problem here.
    
    
```
Initialize dp[] with appropriate initial values

for each node a in graph A's full path:
    for each node b in graph B:
        if b's parent has been visited in graph A's path:
            dp[b] = max(dp[b], dp[b's parent] + weight of (b's parent to b))

Find the node b_last with the maximum value in dp[]
Backtrack from b_last to find the actual path in graph B
```

 

In [1]:
import pysam
import kdp
import numpy as np
bam = pysam.AlignmentFile("test/NA24385.chr20.bam")
ref = pysam.FastaFile("/Users/english/code/references/grch38/GRCh38_1kg_mainchrs.fa")

In [19]:
#chrom, start, end = "chr20", 28797607-50, 28797607+120+50 # deletion
chrom, start, end = "chr20", 20827970, 20827980

In [3]:
refseq = ref.fetch(chrom, start, end)
reads = bam.fetch(chrom, start, end)

```
M	BAM_CMATCH	0
I	BAM_CINS	1
D	BAM_CDEL	2
N	BAM_CREF_SKIP	3
S	BAM_CSOFT_CLIP	4
H	BAM_CHARD_CLIP	5
P	BAM_CPAD	6
=	BAM_CEQUAL	7
X	BAM_CDIFF	8
B	BAM_CBACK	9
```

In [49]:
all_cov = 0
all_ks = {} # readname: Haplotype
chrom, start, end = "chr20", 20827970, 20827980
BUFFER = 500
for column in bam.pileup(chrom, start - BUFFER, end + BUFFER, truncate=True):
    # Check for deletions

    for read in column.pileups:
        # Guard against partial alignments which mess up the kfeat 
        # Will revisit when I can turn a Haplotype into a single-path graph
        if not ((read.alignment.reference_start < start) and (read.alignment.reference_end > end)):
            continue
        all_cov += 1
        # Only consider things greater than 20bp
        if abs(read.indel) < 20:
            continue
        if read.indel > 20:  # Insertion greater than 20 bp
            seq = read.alignment.query_sequence[read.query_position:read.query_position + read.indel]
            m_hap = kdp.Haplotype(kdp.seq_to_kmer(seq, 4), read.indel, 1)
        elif read.indel < -20:  # Deletion greater than 20 bp
            m_start = column.reference_pos - start
            m_end = m_start + abs(read.indel)
            m_hap = kdp.Haplotype(-kdp.seq_to_kmer(refseq[m_start: m_end], 4), read.indel, 1)
        if read.alignment.query_name not in all_ks:
            all_ks[read.alignment.query_name] = m_hap
        else:
            all_ks[read.alignment.query_name] += m_hap
# Region coverage
reg_cov = all_cov / (end - start)

len(all_ks), reg_cov
all_ks['reference'] = kdp.Haplotype(kdp.seq_to_kmer("", 4), 0, reg_cov - len(all_ks))

In [68]:
column.n

11

In [46]:
len(all_ks)

11

Try to make two groups from just the alts:

If our two groups don't separate, we either have HOMALT or REFHET.
Either way, the whole group of alts becomes hap2.
We then decide if we want hap1 to be REF or HOMALT based on the proportion ( 85%+ becomes HOMALT )

If they do separate, then we just choose one to be hap1 and one to be hap2

In [5]:
# For this example region, we know there's just a single hap
hap1 = kdp.Haplotype.new(4)
hap1.kfeat = np.mean([_.kfeat for _ in all_ks.values()], axis=0)
hap1.size = np.mean([_.size for _ in all_ks.values()])
hap1.n = np.mean([_.n for _ in all_ks.values()])
hap1.coverage = len(all_ks)

In [32]:
# Figuring out how to handle compound variants, I'll try to make upto two haps.
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 2, random_state = 0)

In [ ]:
# if number of variant reads is approximately 50%, we expect only 1 alternate group
# if the number of variant reads is like ≥80%, we expect possibly a compound het, or a hom 
# if the number of variant reads is approximately 0%, we expect reference homozygous across

In [59]:
y = [1 if _ != 'reference' else 0 for _ in all_ks.keys()]

In [63]:
weight = [_.coverage for _ in all_ks.values()]
weight

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [64]:
group = kmeans.fit_predict([_.kfeat for _ in all_ks.values()], sample_weight=weight)

# if t
grps = list(zip(, all_ks.keys()))
grps

[(0, 'm54329U_201103_231616/14680702/ccs'),
 (0, 'm54329U_201103_231616/146802986/ccs'),
 (0, 'm54329U_201103_231616/81789676/ccs'),
 (0, 'm54329U_201103_231616/64227876/ccs'),
 (0, 'm54329U_201103_231616/6490175/ccs'),
 (0, 'm54329U_201103_231616/127994120/ccs'),
 (0, 'm54329U_201103_231616/88409094/ccs'),
 (0, 'm54329U_201103_231616/58982504/ccs'),
 (0, 'm54329U_201103_231616/105840958/ccs'),
 (0, 'm54329U_201103_231616/32441695/ccs'),
 (1, 'reference')]

In [ ]:
len(all_ks)

In [69]:
base = "TGTGTGCTGAGTCCAGCTCAAGTCCCTTGGTTCCCACTGCTGCTAAGCATGCACG"; comp = "GTGTGCGTGTGCCACCATGCCTCCTTTTCCCACCGCTTTAGTGATGGATGCTGG"

In [78]:
target = 50
outs = [-50, 50]
sorted([(abs(target - i),j) for j,i in enumerate(outs)])

[(0, 1), (100, 0)]

In [75]:
target = 50
outs = [-50, 50]
sorted([(abs(target - i),j) for j,i in enumerate(outs)])

[(0, 1), (100, 0)]

In [80]:
def pareto_optimal_tuples(tuples):
    pareto_optimal = []
    dominated_indices = set()
    for i, (x1, y1, z1) in enumerate(tuples):
        if i in dominated_indices:
            continue
        is_pareto = True
        for j, (x2, y2, z2) in enumerate(tuples):
            if i != j and (x2 >= x1 or y2 >= y1 or z2 >= z1):
                if x2 > x1 or y2 > y1 or z2 > z1:
                    is_pareto = False
                    dominated_indices.add(i)
                    break
                else:
                    dominated_indices.add(j)
        if is_pareto:
            pareto_optimal.append((x1, y1, z1))
    return pareto_optimal

# Example usage:
tuples = [(1, 2, 3), (4, 5, 6), (3, 6, 9), (7, 8, 8)]
pareto_optimal = pareto_optimal_tuples(tuples)
print("Pareto optimal tuples:", pareto_optimal)


Pareto optimal tuples: []


In [104]:
import math
def genotyper(totCov, altCov, priors=None):
    """
    Given total coverage and altCoverage, try to calculate how many copies
    of the alt are at the position (ref/het/hom) and a quality score
    returns two lists
    - probabilities of 0, 1, or two copies of the allele at the location
    - phred-scaled quality scores of those probs
    """
    #We have no information.. should give up
    if totCov == 0:
        return None

    # previously had avgCov
    if priors is None:
        priors = [0.05, 0.5, 0.95]

    # if len(priors) != 3: # raise exception?

    def log_choose(n, k):
        """ swap for efficiency if k is more than half of n """
        r = 0.0
        if k * 2 > n:
            k = n - k

        for d in range(1, k + 1):
            r += math.log(n, 10)
            r -= math.log(d, 10)
            n -= 1

        return r

    total = totCov  # refCoverage + altCoverage if avgCov is None else avgCov
    alt = altCov  # int(spot.tags["szCount"])
    non_alt = total - alt

    gtList = []

    comb = log_choose(total, alt)
    for p_alt in priors:
        gtList.append(comb + alt * math.log(p_alt, 10) + non_alt * math.log(1 - p_alt, 10))

    return gtList

In [129]:
for i in range(1, 30):
    for j in range(1, i):
        for k in genotyper(i,j):
            print(i, j, k, int(10 * math.log(1-k)))
        print()

2 1 -1.0222763947111522 7
2 1 -0.30102999566398114 2
2 1 -1.0222763947111517 7

3 1 -0.8684615303666231 6
3 1 -0.425968732272281 3
3 1 -2.147215131319451 11

3 2 -2.147215131319452 11
3 2 -0.425968732272281 3
3 2 -0.8684615303666227 6

4 1 -0.7657991884694755 5
4 1 -0.6020599913279623 4
4 1 -3.323306390375132 14

4 2 -1.8684615303666232 10
4 2 -0.42596873227228105 3
4 2 -1.868461530366622 10

4 3 -3.3233063903751328 14
4 3 -0.6020599913279623 4
4 3 -0.765799188469475 5

5 1 -0.6911655701725713 5
5 1 -0.806179973983887 5
5 1 -4.527426373031056 17

5 2 -1.6688891754614188 9
5 2 -0.5051499783199058 4
5 2 -2.9476427764142468 13

5 3 -2.947642776414247 13
5 3 -0.5051499783199058 4
5 3 -1.668889175461418 9

5 4 -4.527426373031058 17
5 4 -0.806179973983887 5
5 4 -0.6911655701725709 5

6 1 -0.6342607188360988 4
6 1 -1.0280287236002434 7
6 1 -5.749275122647411 19

6 2 -1.51507431111689 9
6 2 -0.630088714928206 4
6 2 -4.072581513022546 16

6 3 -2.668889175461419 12
6 3 -0.505149978319906 4
6 3 -

In [118]:
10**((5.727597453449266)/10)

3.7390368499084894